## Mycoast Pollution Risk Operational Model

This document will show users of [MyCoastLCS](https://gitlab.ecosystem-modelling.pml.ac.uk/ricardo.torres/mycoast-ftle.git) how to setup a first example of the tool using lagrangian simulations performed with [Pylag](https://gitlab.ecosystem-modelling.pml.ac.uk/PyLag/PyLag) (both codes require an account to PML's gitlab server, please sign up [here](https://www.pml.ac.uk/Modelling_at_PML/Access_Code)). Click on Other in the Model Code choice and specify PyLag in the comment box. PyLag is not needed to use MyCoastLCS however some changes to the code might be required to use a different lagrangian particle simulator. 

This notebook will guide you through the steps to track particles from source point locations, and to calculate LCS fields from a pylag (or other lagrangian model) simulation that includes a gridded release of particles.

Simulations can be backwards or forwards and for any number of ensembles and integration times. The direction and integration time will determine the interpretation of the results, whether this is following particle pathways in time, backtracking to identify likely source locations, or displaying attractors or repelling barriers to lagrangian transport.

For more information refer to the [LCS tutorial ](https://shaddenlab.berkeley.edu/uploads/LCS-tutorial/) and the documentation for __MyCoastLCS__.

This notebook runs with the code provided [here](https://drive.google.com/drive/folders/1qfMO5PpzsDURat8wsYaXvvvlSP20I3xy?usp=sharing), which should be decompressed in the ```$HOME``` directory ```~/MyCOASTpollutiontool_example``` with the following folder structure:


  ```
  .
├── input
│   ├── bathingareas_kmsq.csv
│   ├── ftle_initialpositions.dat
│   ├── ftle.json
│   ├── grid_metrics.nc
│   ├── pylag_fvcom.cfg
│   ├── shapefiles
│   │   ├── bovisand.cpg
│   │   ├── bovisand.dbf
│   │   ├── bovisand.prj
│   │   ├── bovisand.shp
│   │   ├── bovisand.shx
│   │   ├── cawsand.cpg
│   │   ├── cawsand.dbf
│   │   ├── cawsand.prj
│   │   ├── cawsand.shp
│   │   ├── cawsand.shx
│   │   ├── crownhill.cpg
│   │   ├── crownhill.dbf
│   │   ├── crownhill.prj
│   │   ├── crownhill.shp
│   │   ├── crownhill.shx
│   │   ├── firestone.cpg
│   │   ├── firestone.dbf
│   │   ├── firestone.prj
│   │   ├── firestone.shp
│   │   ├── firestone.shx
│   │   ├── tinside.cpg
│   │   ├── tinside.dbf
│   │   ├── tinside.prj
│   │   ├── tinside.shp
│   │   └── tinside.shx
│   ├── tamar_v2_0001.nc
│   ├── tamar_v2_obc.dat
│   ├── triggerpoints
│   │   ├── 2017triggers.csv
│   │   ├── CSO_ENGLAND2019.csv
│   │   ├── CSO_PLYMOUTHSOUND_TRIGGERPOINTS.csv
│   │   ├── CSO_TAMAR_TRIGGERPOINTS.csv
│   │   ├── CSOtriggerpoints.ipynb
│   │   └── Rainfall
│   │       ├── 2017
│   │       │   ├── annualRF_2017.nc
│   │       │   ├── fail.csv
│   │       │   ├── fixed.csv
│   │       │   └── missing.csv
│   │       └── 2019
│   │           ├── annualRF_2019.nc
│   │           ├── fail.csv
│   │           ├── fixed.csv
│   │           └── missing.csv
│   └── wrfout_001.nc
├── MyCOASTpollutiontool_example.ipynb
├── simulations
└── tables
    ├── bovisand.csv
    ├── cawsand.csv
    ├── firestone.csv
    └── tinside.csv
```


running the script  will generate a new output folder: Exp01. You can change this in the simname varibale below. Example input files are provided [in this google drive](https://drive.google.com/drive/folders/1qfMO5PpzsDURat8wsYaXvvvlSP20I3xy?usp=sharing) , however FVCOM and WRF files for todays date can be downloaded from the following THREDDS servers:

[FVCOM] (https://data.ecosystem-modelling.pml.ac.uk/thredds/catalog/mycoast-all-files/Model/TAMAR_ESTUARY_FORECAST_PHY_001/tamar_estuary_forecast_phy_001_hourly_t_s_u_v_ssh/today_forecast/catalog.html)
 

[WRF] (https://data.ecosystem-modelling.pml.ac.uk/thredds/catalog/mycoast-all-files/Model/TAMAR_ESTUARY_FORECAST_WIND_001/tamar_estuary_coast_forecast_wind_001_hourly/today_forecast/catalog.html)


This notebook guides the user in running PyLag and the MyCOASTLCS tool to generate predictions of bathing water quality. Bathing water quality predictions are based on a combination of dispersal trajectories of particles representing pollutants from point source locations (in this case Combined Sewage Overspill sites),and the formation of Lagrangian Coherent Structures, which may act as a barrier to particle transport.

This example is run on an FVCOM grid, however [PyLags documentation](https://pylag.readthedocs.io/en/latest/) shows how to setup for running using models defined on an [Arawaka A grid](https://pylag.readthedocs.io/en/latest/examples/arakawa_a_forward_tracking.html).

## Pre-requisites

The example below assumes you are running inside a Linux environment. These instructions have only been tested in systems running Fedora 29 and 33.  

## Installing the software

Prior to running this example, Pylag and the MyCOAST tool must be installed in a conda environment on the users' machine. To do this follow the steps below:

If not already installed, install [miniconda](https://conda.io/projects/conda/en/latest/user-guide/install/linux.html) to a location of your choosing. Then, activate Conda, ensure conda is up to data, and add the channels conda-forge, geo-down-under and JimClark. The channel geo-down-under is needed for one of PyLag’s dependencies. The channel JimClark is a temporary distribution channel for PyLag. For example:

```bash
$ source /opt/miniconda/miniconda3/bin/activate
$ conda update conda
$ conda config --append channels conda-forge
$ conda config --append channels geo-down-under
$ conda config --append channels JimClark
```

The above code assumes miniconda3 was installed into the directory ```/opt/miniconda```, once the appropriate write permissions have been set. The default behaviour is to install miniconda3 into your home directory. This is, of course, also fine.


### Installing Pylag

With miniconda3 installed and configured, create a new environment in which to install PyLag using the following commands:

```bash
$ conda create -n pylag python=3.8
$ conda activate pylag
```

then install pylag

```bash
(pylag) $ conda install -n pylag -c JimClark pylag
```

To test that PyLag has been correctly installed, type:

```bash
(pylag) $ python -c "import pylag"
```

which should exit without error. When installing PyLag with conda, all the dependencies should have been resolved, and you should have PyFVCOM installed. 



Alternatively, Pylag can be installed from PML's repository using the following instructions: 

```bash
$ conda install conda-build -n pylag
$ git clone https://gitlab.ecosystem-modelling.pml.ac.uk/PyLag/PyLag.git
$ cd Pylag
$ git pull
$ conda build . 
$ conda install -n pylag --use-local pylag
$ pip install  . 
```

To generate the documentation from the developer branch you need to download the example files from [here](https://drive.google.com/drive/folders/15UX7Y9JnuLpnPAz700mzmzd917nTClxR?usp=sharing). 

To finally generate the documentation ... 

```bash
$ cd doc; make html
```

### Installing the MyCoast tool

With SSH access setup, you can clone the MyCoastLCS repository using the following commands:

```bash
$ mkdir -p $HOME/mycoast_code/mycoastlcs && cd $HOME/code/git/FTLE
$ git clone https://gitlab.ecosystem-modelling.pml.ac.uk/ricardo.torres/mycoast-ftle.git>
```
and then swap to branch 

```bash
$ git checkout --track origin/T_direction_styling
``` 

When working within conda and pylag environment activated type:

```bash
$ python -m pip install .
```

### Additional python packages required 

Other Python packages that do not come standard in a miniconda installation, pylag or MyCoastLCS but that are needed to run this example are:

```bash
$ conda install -c conda-forge jupyter jupyter-lab -n pylag
$ conda install -c conda-forge cartopy -n pylag
$ conda install dask -n pylag
$ conda install holoviews -n pylag
$ conda install natsort -n pylag
$ conda install -c conda-forge matplotlib -n pylag
$ conda install -c conda-forge cmocean -n pylag
$ conda install pandas -n pylag
```




In [ ]:
import pandas as pd
from datetime import date,datetime,timedelta
import numpy as np
from math import radians, cos, sin, asin, sqrt
import netCDF4 as nc
import datetime
import os 
from os.path import expanduser

###this example uses WRF and FVCOM files for 22nd May 2021

#haversine equation function for calculating the nearest WRF grid cell for each CSO site
def distance(lat1, lon1, lat2, lon2):  
    p = 0.017453292519943295  #Pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a)) #2*R*asin..

#cores to run pylag on (if running parallel)
cores = 4
print('pylag simulation running on ',cores,' cores')

#if the example directory is saved anywhere other than your hom dirctory,
#give the path here
home = expanduser("~")
mycoast = f'{home}/MyCOASTpollutiontool_example'
os.chdir(mycoast)
wd = os.getcwd()
print('working directory: ',wd,'\n')
input_dir = f'{wd}/input'
print('input directory: ',input_dir,'\n')

simname = 'Exp01'

#create a new run directory in the simulations folder
simulation_dir = f"{wd}/simulations/{simname}"
print('simulation directory: ',simulation_dir,'\n')

try:
    os.makedirs(simulation_dir)
except FileExistsError:
    pass

#make the new directory writeable
os.chmod(simulation_dir,0o755)
#path to daily FVCOM file
FVCOMfile = f'{input_dir}/tamar_v2_0001.nc'

#path to bathing site shapefiles
shp_dir = f'{input_dir}/shapefiles'

#CSO triggerpoint file
#a jupyter notebook to guide you through creating a trigger point csv from 
#daily rainfall over a year (WRF output) and CSO annnual overspill data
# is available in ~/{example_folder}/input/triggerpoints/

CSOfile=f'{input_dir}/triggerpoints/CSO_PLYMOUTHSOUND_TRIGGERPOINTS.csv'

#get a copy of the default config
default_cfg = f'{input_dir}/pylag_fvcom.cfg'

#grid metrics
grid_metrics_file_name = f'{input_dir}/grid_metrics.nc'

#seedfile for ftle
ftle_seedfile = '/data/proteus1/scratch/moja/MyCOAST/mycoast_code/operational_model/resources/ftle_initialpositions.dat'


##the grid metrics file and ftle seed file have been provided in the example documentation
#however if you want to make your own, set these to false
gridmetrics = True
ftleseeds = True 
#import bathing area size csv data
bathingareas = f'{input_dir}/bathingareas_kmsq.csv'

tables = f'{mycoast}/tables'

#ftle json
ftlejson =f'{input_dir}/ftle.json'


In [ ]:
print('Getting daily rainfall data from WRF')

#read in daily WRF output and calculate daily rainfall at each of the CSO sites
##read in todays WRF data
WRFpath = f'{input_dir}/wrfout_001.nc'
print('WRF files read in from: ',WRFpath)

#read in the grid from the annual WRF file
WRFfile = nc.Dataset(WRFpath, 'r')
#read in coordinates
WRFlat = WRFfile.variables['XLAT'][:][0,:,:]
WRFlon = WRFfile.variables['XLONG'][:][0,:,:]
#store as columns
listlon = np.reshape(WRFlon, ((WRFlon.shape[0]*WRFlon.shape[1]), 1))
listlat = np.reshape(WRFlat, ((WRFlat.shape[0]*WRFlat.shape[1]), 1))
listlon = np.ma.filled(listlon)
listlat = np.ma.filled(listlat)

#read in rainfall
dailyrain =  WRFfile.variables['RAINNC'][:][:,:,:]

#WRF has a hindcast and forecast and rain is accumulated from the first timestep. Therefore to get the daily rainfall we need to 
#subtract rainfall, at the 13th timestep (12:00 yesterday) from the 21st timestep (12:00) today

dailyrain = dailyrain[21,:,:] - dailyrain[13,:,:]

#read in the CSO file to a pandas dataframe
df=pd.read_csv(CSOfile, sep=',',header=0)

CSOlon = np.asarray(df['lon'])
CSOlat = np.asarray(df['lat'])
TP = np.asarray(df['TriggerPoint'])

#read in the grid from the FVCOM file
file = nc.Dataset(FVCOMfile, 'r')
FVCOMlat = file.variables['latc'][:]
FVCOMlon = file.variables['lonc'][:]


#create empty arrays for CSO FVCOM points
CSOFVCOMlon = np.zeros(len(df))
CSOFVCOMlat = np.zeros(len(df))

print('determine CSO overspill in Plymouth Sound catchment')
#loop through the CSO sites
for x in range(len(df)):
        #search for the coordinates of the CSO site
    dis = np.zeros(len(FVCOMlon))
    qlat = CSOlat[x]
    qlon = CSOlon[x]
    
    #find the index of the node (lat/lon) nearest to the CSO site
    for xx in range(len(FVCOMlon)):
        dis[xx] = distance(qlat, qlon, FVCOMlat[xx], FVCOMlon[xx])
   
    mindex = np.argmin(dis)
    
    for xx in range(len(FVCOMlon)):
        if FVCOMlon[xx] == FVCOMlon[mindex] and FVCOMlat[xx] == FVCOMlat[mindex]:
            CSOFVCOMlon[x] = FVCOMlon[xx]
            CSOFVCOMlat[x] = FVCOMlat[xx]

        else:
            continue


WRFindex_lon = np.zeros(len(df))
WRFindex_lat = np.zeros(len(df))
spill_bool = np.zeros(len(df))

#loop through the CSO sites
for x in range(len(df)):
        #search for the coordinates of the CSO site
    dis = np.zeros(len(listlon))
    qlat = CSOlat[x]
    qlon = CSOlon[x]
    for j in range(len(listlon)):
        dis[j] = distance(qlat, qlon, listlat[j], listlon[j])
    mindex = np.argmin(dis)

    
    for i in range(WRFlon.shape[0]):
        for j in range(WRFlon.shape[1]):
            if WRFlon[i,j] == listlon[mindex] and WRFlat[i,j] == listlat[mindex]:
                WRFindex = [i,j]
                WRFindex_lon[x] = i
                WRFindex_lat[x] = j
            else:
                continue


    #get rain at site location
    rain = dailyrain[WRFindex[0],WRFindex[1]]
    
    if rain >= TP[x]:
        spill_bool[x] = 1
    else:
        spill_bool[x] = 0

spill_bool = spill_bool.astype(bool)        

print('CSO release: ',spill_bool)

In [ ]:
print('Creating the seedile for CSO overspill')

#create the seedfile

from datetime import timedelta, date
from pylag.processing.coordinate import lonlat_from_utm, utm_from_lonlat
from pylag.processing.input import create_initial_positions_file_single_group, create_initial_positions_file_multi_group
from pylag.processing.release_zone import create_release_zone

#import spills info for weighted particle release and calculate weighted metric
spills = np.asarray(df['Spills'])
spilldur = np.asarray(df['Spill_Duration'])
weight = spilldur/spills
del(spills)
del(spilldur)

#set number of particles released from each ensemble prior to weighting
nbasic = 100
radius = 0  # Release zone radius in meter
depth = 0.0  # Depth of particles
print('unweighted number of  particles per ensemble: ',nbasic)

numspillsites = np.sum(spill_bool)
print('number of active CSO overspill sites in study domain: ',numspillsites)

 #create empty arrays for spill locations and site specific weights
spill_lon = np.zeros(len(df))
spill_lat = np.zeros(len(df))
spill_weight = np.zeros(len(df))


for x in range(len(df)):
    if spill_bool[x] == True:
        spill_lon[x] = CSOFVCOMlon[x]
        spill_lat[x] = CSOFVCOMlat[x]
        spill_weight[x] = weight[x]
    else:
        spill_lon[x] = np.nan
        spill_lat[x] = np.nan
        spill_weight[x] = np.nan

#remove NaNs to give list of spill locations for date
spill_lon = spill_lon[~np.isnan(spill_lon)]
spill_lat = spill_lat[~np.isnan(spill_lat)]
spill_weight = spill_weight[~np.isnan(spill_weight)]

relsites = len(spill_lon)
siteno = len(df)
zpos = len(spill_lon)

# Output filename
seedfile = f"{simulation_dir}/PylagCSOseed_{simname}.dat"

# The group ID of this particle set (CSO sites have the ID number 1)
group_id = 1
release_zones = []

###### if no particles are released by any CSO site we will release some ransom extra particles just to keep things ticking over:
is_empty = spill_lon.size == 0.
if is_empty is True:
    extra_lon = -4.15  #random location
    extra_lat = 50.32  #in domain
    extrap = 1
    group_id = 0
    extrax,extray,zone  = utm_from_lonlat(extra_lon,extra_lat) 
    release_zone = create_release_zone(group_id, radius,np.asarray([extrax[0],extray[0]]), int(extrap), depth,
                                           random=True)
    release_zones.append(release_zone)
    create_initial_positions_file_multi_group(seedfile, release_zones)
    
    #for parallel runs, the number of particles needs to be divisible by the number of cores. Read in the initial positions file and check the number of particles.
    # if this is not divisible by the number of cores create a new release zone with extra particles to make up the number. we will filter out these particles 
    #in post processing
    
    sf = pd.read_csv(seedfile, sep=" ",header=0)
    num_p = sf.shape[0]

    if cores % num_p == 0 and cores <= num_p == True:
        print('number of particles are divisble by nymber of cores')
        tot_p = num_p
        print(f'Total particles : {tot_p}')
    else:
        if cores > num_p:
            print('fewer particles than cores - adding extra particles to initial positions file')
            extrap = cores - num_p
        else:
            print('number of particles are not divisble by number of cores - adding extra particles to initial positions file')
            extrap = (np.ceil(num_p/cores)*cores)-num_p
        print('adding ',extrap,' particles')
        tot_p = (num_p + extrap)
        print(f'Total particles : {tot_p}')
        group_id = 0

        #random location in domain
        extra_lon = -4.15
        extra_lat = 50.32
        extrax,extray,zone  = utm_from_lonlat(extra_lon,extra_lat) 
        release_zone = create_release_zone(group_id, radius,np.asarray([extrax[0],extray[0]]), int(extrap), depth,
                                           random=True)
        release_zones.append(release_zone)
        create_initial_positions_file_multi_group(seedfile, release_zones)


else:
    #convert to utm
    spillx, spilly, epsg_code = utm_from_lonlat(spill_lon, spill_lat)
    n = len(spilly)
    # does it matter if there are repeated nodes? probably not
    for x in range(len(spill_lon)):
        release_zone = create_release_zone(group_id, radius,np.asarray([spillx[x],spilly[x]]), int(nbasic*spill_weight[x]), depth,
                                           random=True)
        release_zones.append(release_zone)


    # Write data to file
    create_initial_positions_file_multi_group(seedfile, release_zones)
    

    #for parallel runs, the number of particles needs to be divisible by the number of cores. Read in the initial positions file and check the number of particles.
    # if this is not divisible by the number of cores create a new release zone with extra particles to make up the number. we will filter out theseparticles in post processing
    sf = pd.read_csv(seedfile, sep=" ",header=0)
    num_p = sf.shape[0]
    print('number of CSO particles released per hour in simulation:',' ',num_p)

    
    if cores % num_p == 0 and cores <= num_p == True:
        print('number of particles are divisble by nymber of cores')
        tot_p = num_p
        print(f'Total particles : {tot_p}')
    else:
        if cores > num_p:
            print('fewer particles than cores - adding extra particles to initial positions file')
            extrap = cores - num_p
        else:
            print('number of particles are not divisble by number of cores - adding extra particles to initial positions file')
            extrap = (np.ceil(num_p/cores)*cores)-num_p
        print('adding ',extrap,' particles')
        tot_p = (num_p + extrap)
        print(f'Total particles per simulation: {tot_p}')
        ppd = tot_p*12
        print(f'Total particles per day: {ppd}')
        group_id = 0

            #random location in domain
        extra_lon = -4.15
        extra_lat = 50.32
        extrax,extray,zone  = utm_from_lonlat(extra_lon,extra_lat) 
        release_zone = create_release_zone(group_id, radius,np.asarray([extrax[0],extray[0]]), int(extrap), depth,
                                       random=True)
        release_zones.append(release_zone)
        create_initial_positions_file_multi_group(seedfile, release_zones)



In [ ]:
#set up pylag

################################
#  create the grid metrics file  #
##################################

# a grid metrics file exists in the resources folder but this section can create a new one if needed

if gridmetrics == False:
    ###fvcom###
    print('creating grid metrics file')
    from pylag.grid_metrics import create_fvcom_grid_metrics_file
    # The file listing the location of open boundary nodes
    obc_file_name = f'{input_dir}/tamar_v2_obc.dat'

    # The name of the output file
    grid_metrics_file_name = f'{input_dir}/grid_metrics.nc'

    # Generate the file
    create_fvcom_grid_metrics_file(FVCOMfile, obc_file_name, grid_metrics_file_name)

    # read mesh
    varsinfvcom = ['bathy', 'x', 'y', 'lon', 'lat', 'triangles']
    fvcom = dict.fromkeys(varsinfvcom)

    # read mesh variables
    with nc.Dataset(grid_metrics_file_name) as fvcomdata:
        fvcom['bathy'] = fvcomdata.variables['h'][:]
        fvcom['x'] = fvcomdata.variables['x'][:]
        fvcom['y'] = fvcomdata.variables['y'][:]
        fvcom['lon'] = fvcomdata.variables['longitude'][:]
        fvcom['lat'] = fvcomdata.variables['latitude'][:]
        fvcom['triangles'] = fvcomdata.variables['nv'][:].transpose()

##########################
#    write the config    #
##########################

import configparser
cf = configparser.ConfigParser()
cf.read(default_cfg)

print('writing the config file for pylag CSO simulation')
# set the directories and output file name 
cf.set('GENERAL', 'in_dir',simulation_dir)
cf.set('GENERAL', 'out_dir',simulation_dir)
cf.set('GENERAL', 'output_file',f'pylag_CSO_run_{simname}')

 # Set the time to start simulations 
Date = datetime.datetime(2021,6,21)
Date = datetime.datetime.combine(Date, datetime.datetime.min.time())
date_string = Date.strftime('%Y-%m-%d %H:%M:%S')
delta = timedelta(hours = 24)
Date = Date - delta
date_string = Date.strftime('%Y-%m-%d %H:%M:%S')

cf.set('SIMULATION', 'start_datetime',date_string)
print('Start time: {}'.format(cf.get('SIMULATION', 'start_datetime')))

#Set the time to end simulations if not using an ensemble approach
# cf.set('SIMULATION', 'end_datetime','2013-01-06 22:00:00')
#if using an ensemble approach, set this to blank
cf.set('SIMULATION', 'end_datetime','')
print('End time: {}'.format(cf.get('SIMULATION', 'end_datetime')))

# Specify that this is a forward tracking experiment
cf.set('SIMULATION', 'time_direction','forward')
print('Time direction: {}'.format(cf.get('SIMULATION', 'time_direction')))

#set the path to the initial positions file
cf.set('SIMULATION','initial_positions_file', seedfile)
print('Seedfile: {}'.format(cf.get('SIMULATION', 'initial_positions_file')))

# We will do an ensemble run
# for this test we will restrict the ensemble to 24 instances 
cf.set('SIMULATION', 'number_of_particle_releases','24')
print('Number of particle releases: {}'.format(cf.get('SIMULATION', 'number_of_particle_releases')))

# set duration of each ensemble release  
cf.set('SIMULATION', 'duration_in_days','1')
print('Duration of each release (hours): {}'.format(cf.getfloat('SIMULATION', 'duration_in_days')*24))

#set the particle release interval
cf.set('SIMULATION', 'particle_release_interval_in_hours','1')
print('Particle release interval: {}'.format(cf.get('SIMULATION', 'particle_release_interval_in_hours')))

# Use depth restoring, and restore particle depths to the ocean surface
cf.set('SIMULATION', 'depth_restoring','True')
print('Use depth restoring: {}'.format(cf.get('SIMULATION', 'depth_restoring')))
print('Restore particles to a depth of: {} m'.format(cf.get('SIMULATION', 'fixed_depth')))

# Specify that we are working with and unstructured FVCOM in cartesian coordinates
cf.set('OCEAN_CIRCULATION_MODEL', 'name','FVCOM')
cf.set('OCEAN_CIRCULATION_MODEL', 'coordinate_system','cartesian')
print('Model name: {}'.format(cf.get('OCEAN_CIRCULATION_MODEL', 'name')))
print('Coordinate system: {}'.format(cf.get('OCEAN_CIRCULATION_MODEL', 'coordinate_system')))

# Set the location of the grid metrics and input files
cf.set('OCEAN_CIRCULATION_MODEL', 'data_dir',input_dir)
print('Data directory: {}'.format(cf.get('OCEAN_CIRCULATION_MODEL', 'data_dir')))

cf.set('OCEAN_CIRCULATION_MODEL', 'grid_metrics_file', grid_metrics_file_name)
print('Path to grid metrics file: {}'.format(cf.get('OCEAN_CIRCULATION_MODEL', 'grid_metrics_file')))

# Set fvcom output file name stem
# it is important that the non-stem files can be numerically ordered by wildcard 
cf.set('OCEAN_CIRCULATION_MODEL', 'data_file_stem', 'tamar_v2_0') 
print('File name stem of input files: {}'.format(cf.get('OCEAN_CIRCULATION_MODEL', 'data_file_stem')))

cf.set('NUMERICS', 'iterative_method', 'AdvDiff_Milstein_3D') 

# Save a copy in the simulation directory
with open(f"{simulation_dir}/pylagcso{simname}.cfg", 'w') as config:
    cf.write(config)
pcfg = f'{simulation_dir}/pylagcso{simname}.cfg'    

###################
#  setup the run  #
###################

import subprocess
import sys
import stat

# Change to the run directory and launch
os.chdir(simulation_dir)
print(f'running pylag for CSO in {simulation_dir}')

# write submit script 
with open(f"csorun_{simname}.sh", "w") as f:
    f.write("#!//usr/bin/env bash\n")
    f.write("set dir .\n")
 #  f.write(f"python -m pylag.main -c {pcfg}\n") # for serial applications
    f.write(f"mpiexec -np {cores} python -m pylag.parallel.main -c {pcfg}\n") # for parallel applications
    
os.chmod(f"csorun_{simname}.sh", stat.S_IRWXU)




In [ ]:
#run pylag

try:
    subprocess.call([f'./csorun_{simname}.sh'])
except:
    print('Run failed.')
    pass
print('pylag for CSO ran sucessfully')

##you can follow the progress of the pylag run by typing tail -f pylag_out.log in the simulation directory

In [ ]:
#### create plot of cso particles at midday
print('plotting CSO particles at midday ')
from pylag.processing.ncview import Viewer
from datetime import timedelta

datelist = []
time_indexlist = []
lonslist=[]
latslist = []
lonspaths_list = []
latspaths_list = []
pidlist= []


tof = 25
#extract particle positions on midday on the quary day. we will pull from each ensemble on midday of the query day
for x in range(1,25):
    tof = tof-1
    
    filename = f'{simulation_dir}/pylag_CSO_run_{simname}_{x}.nc'
    print('ensemble ',simname,'_',x)
    print('Time of flight: ',tof)
    # Time of flight
    time_of_flight = timedelta(hours=tof)

    # Dataset holding particle positions
    viewer = Viewer(filename, time_rounding=900)

    # Get time index
    date = viewer.date[0] + time_of_flight
    datelist.append(date)
    time_index = viewer.date.tolist().index(date)
    time_indexlist.append(time_index)

    # Convert positions into lons/lats
    lons, lats = lonlat_from_utm(viewer('x')[time_index, :].squeeze(),
                             viewer('y')[time_index, :].squeeze(), epsg_code='32630')
    lonslist.append(lons)
    latslist.append(lats)
    
    # Convert all pathline coordinates into lons/lats
    x = viewer('x')[:time_index, :].squeeze()
    y = viewer('y')[:time_index, :].squeeze()
    arr_size = x.shape
    lons_paths, lats_paths = lonlat_from_utm(x.flatten(), y.flatten(), epsg_code='32630')
        
    lonspaths_list.append(lons_paths.reshape(arr_size))
    latspaths_list.append(lats_paths.reshape(arr_size))
    
    pid = viewer('group_id')[:]
    pidlist.append(pid)



In [ ]:
#extract CSO points (pid = 1)
csolons = []
csolats = []

for i in range(len(lonslist)):
    boolarr = pidlist[i] == 1
    Lon = lonslist[i]
    Lat = latslist[i]
    csolons.append(Lon[boolarr])
    csolats.append(Lat[boolarr])
    
import cartopy.crs as ccrs
import warnings

from pylag.processing.plot import FVCOMPlotter
from pylag.processing.plot import create_figure, colourmap
from matplotlib.lines import Line2D
from netCDF4 import Dataset

warnings.filterwarnings('ignore')

# Read in the bathymetry
ds = Dataset(grid_metrics_file_name, 'r')
bathy = -ds.variables['h'][:]
blanks = np.zeros_like(bathy)
ds.close()
del(ds)


# Create figure - Plymouth Sound
font_size = 15
cmap = colourmap('h_r')


fig, ax = create_figure(figure_size=(26., 26.), projection=ccrs.PlateCarree(), font_size=font_size, bg_color='gray')

# Configure plotter
plotter = FVCOMPlotter(grid_metrics_file_name,
                       geographic_coords=True,
                       font_size=font_size)

xmin = -4.3
xmax = -4.075
ymin = 50.295
ymax = 50.52


plt_lims = np.array([xmin, xmax, ymin, ymax])
extents = plt_lims

# Plot the bathymetry again. We'll overlay pathlines on top of this.
plotter.plot_field(ax, bathy, extents=extents, add_colour_bar=True, cb_label='Depth (m)',
                   vmin=-60., vmax=0., cmap=cmap)


for i in range(len(lonslist)):
# Plot particle final positions
    ax, scatter = plotter.scatter(ax, csolons[i], csolats[i], s=8, color='k', edgecolors='none')
    
# plot CSO sites
for i in range(len(CSOlon)):
    if spill_bool[i] == True:
        ax, scatter = plotter.scatter(ax, CSOlon[i], CSOlat[i], s=70, color='r', edgecolors='k')
    else:
        ax, scatter = plotter.scatter(ax, CSOlon[i], CSOlat[i], s=50, marker = 'x', color='k')



numpar = 0
for i in range(len(csolons)):
    numpar= numpar + len(csolons[i])

plotter.set_title(ax, f'Target: 2021-05-22 12:00:00 n = {numpar}')

import shapefile as shp  # Requires the pyshp package
import matplotlib.pyplot as plt
from matplotlib.pyplot import fill


sf = shp.Reader(f'{shp_dir}/cawsand.shp')
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.fill(x,y,'purple',alpha=0.7)
    
sf = shp.Reader(f'{shp_dir}/bovisand.shp')
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.fill(x,y,'yellow',alpha=0.7)

sf = shp.Reader(f'{shp_dir}/firestone.shp')
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.fill(x,y,'red',alpha=0.7)
    
    
    
sf = shp.Reader(f'{shp_dir}/tinside.shp')
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.fill(x,y,'green',alpha=0.7)
    
legend_elements = [Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='yellow', markeredgecolor= 'yellow', markersize=7, label = 'Bovisand Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='green', markeredgecolor= 'g', markersize=7, label = 'Tinside East'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='red', markeredgecolor= 'r', markersize=7, label = 'Firestone Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='purple', markeredgecolor= 'purple', markersize=7, label = 'Kingsand Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='red', markeredgecolor= 'black', markersize=15, label = 'Active CSOs'),
                      Line2D([0], [0], marker = 'x', color = 'w', markerfacecolor='black', markeredgecolor= 'black', markersize=10, label = 'Inactive CSOs')]


ax.legend(handles=legend_elements, loc='best',framealpha=1)

plt.savefig(f'{simulation_dir}/Tamar_CSO_{simname}.png')
plt.close()
#zoom in on the sound
xmin = -4.23
xmax = -4.1
ymin = 50.295
ymax = 50.38

fig, ax = create_figure(figure_size=(26., 26.), projection=ccrs.PlateCarree(), font_size=font_size, bg_color='gray')

plt_lims = np.array([xmin, xmax, ymin, ymax])
extents = plt_lims

# Plot the bathymetry. We'll overlay particles on top of this.
plotter.plot_field(ax, bathy, extents=extents, add_colour_bar=True, cb_label='Depth (m)',
                   vmin=-60., vmax=0., cmap=cmap)


for i in range(len(lonslist)):
# Plot particle final positions
    ax, scatter = plotter.scatter(ax, csolons[i], csolats[i], s=8, color='k', edgecolors='none')
    
# plot CSO sites
for i in range(len(CSOlon)):
    if spill_bool[i] == True:
        ax, scatter = plotter.scatter(ax, CSOlon[i], CSOlat[i], s=70, color='r', edgecolors='k')
    else:
        ax, scatter = plotter.scatter(ax, CSOlon[i], CSOlat[i], s=50, marker = 'x', color='k')


plotter.set_title(ax, f'Target: 2021-05-22 12:00:00 n = {numpar}')

cawsand = shp.Reader(f'{shp_dir}/cawsand.shp')
for shape in cawsand.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.fill(x,y,'purple',alpha=0.7)
    
bovisand = shp.Reader(f'{shp_dir}/bovisand.shp')
for shape in bovisand.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.fill(x,y,'yellow',alpha=0.7)

firestone = shp.Reader(f'{shp_dir}/firestone.shp')
for shape in firestone.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.fill(x,y,'red',alpha=0.7)
       
tinside = shp.Reader(f'{shp_dir}/tinside.shp')
for shape in tinside.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.fill(x,y,'green',alpha=0.7)
    
legend_elements = [Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='yellow', markeredgecolor= 'yellow', markersize=7, label = 'Bovisand Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='green', markeredgecolor= 'g', markersize=7, label = 'Tinside East'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='red', markeredgecolor= 'r', markersize=7, label = 'Firestone Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='purple', markeredgecolor= 'purple', markersize=7, label = 'Kingsand Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='red', markeredgecolor= 'black', markersize=15, label = 'Active CSOs'),
                      Line2D([0], [0], marker = 'x', color = 'w', markerfacecolor='black', markeredgecolor= 'black', markersize=10, label = 'Inactive CSOs')]


ax.legend(handles=legend_elements, loc='best',framealpha=1)

plt.savefig(f'{simulation_dir}/PlymouthSound_CSO_{simname}.png')
#plt.close()

In [ ]:
print('counting CSO concentrations in Plymouth sound bathing areas')

#count the number of CSO particles in each bathing area and calculate concentration of particles (per km2)
from shapely.geometry import Point, Polygon
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface
import csv
#flatten particle location lists into single numpy arrays
csolons = np.hstack(csolons)
csolats = np.hstack(csolats)
#csolons = np.concatenate(csolons).ravel()
#csolats = np.concatenate(csolats).ravel()
    
bovisandcsocount = 0
tinsidecsocount = 0
firestonecsocount = 0
cawsandcsocount = 0

bovisandpoly = bovisand.shapes() 
tinsidepoly = tinside.shapes() 
firestonepoly = firestone.shapes() 
cawsandpoly = cawsand.shapes() 

for i in range(csolons.shape[0]):
    point=Point(csolons[i],csolats[i])
    if point.within(shape(bovisandpoly)): 
        bovisandcsocount = bovisandcsocount+1

for i in range(csolons.shape[0]):
    point=Point(csolons[i],csolats[i])
    if point.within(shape(tinsidepoly)): 
        tinsidecsocount = tinsidecsocount+1
        
for i in range(csolons.shape[0]):
    point=Point(csolons[i],csolats[i])
    if point.within(shape(firestonepoly)): 
        firestonecsocount = firestonecsocount+1
        
for i in range(csolons.shape[0]):
    point=Point(csolons[i],csolats[i])
    if point.within(shape(cawsandpoly)): 
        cawsandcsocount = cawsandcsocount+1
        

with open(bathingareas) as csv_file:
    bathing_df = csv.reader(csv_file, delimiter=',')
    bathing_df = list(bathing_df)
   
bovikm = bathing_df[1]
bovikm = bovikm[1]
if bovisandcsocount == 0:
    bovisandcsokm = 0
else:
    bovisandcsokm = float(bovisandcsocount)/float(bovikm)

tinskm = bathing_df[2]
tinskm = tinskm[1]
if tinsidecsocount == 0:
    tinsidecsokm = 0
else:
    tinsidecsokm = float(tinsidecsocount)/float(tinskm)

firekm = bathing_df[3]
firekm = firekm[1]
if firestonecsocount == 0:
    firestonecsokm = 0
else:
    firestonecsokm = float(firestonecsocount)/float(firekm)

cawskm = bathing_df[4]
cawskm = cawskm[1]
if cawsandcsocount == 0:
    cawsandcsokm = 0
else:
    cawsandcsokm = float(cawsandcsocount)/float(cawskm)



In [ ]:
##### FTLE and LCS
#create the FTLE seedfile if we dont already have one
if ftleseeds == False:
    print('creating ftle seedfile for pylag')
    #zoom in on the sound
    # this will restrict the plot and distribution of the initial locations to the plymouth sound
    plt_lims = np.array([xmin, xmax, ymin, ymax])

    #create particle initial positions (evenly spaced throughout domain for FTLE run)

    resolution = 250
    x_grid = np.linspace(plt_lims[0], plt_lims[1], num=resolution)
    y_grid = np.linspace(plt_lims[2], plt_lims[3], num=resolution)
    x_grid_2d, y_grid_2d = np.meshgrid(x_grid, y_grid)
    x_pts = x_grid_2d.flatten()
    y_pts = y_grid_2d.flatten()
    print(f'The initial positions for LCS calculations are spaced within the domain: with a resolution of {resolution} \n ')
    x_coords, y_coords, zone = utm_from_lonlat(x_pts, y_pts)
    n = len(y_coords)
    zpos = y_coords*0.0
    
    # The group ID of this particle set
    group_id = '1'
    
    # Output filename
    ftle_seedfile = f"{input_dir}/ftle_initialpositions.dat"

    # Write data to file
    create_initial_positions_file_single_group(ftle_seedfile, n, group_id, x_coords, y_coords, zpos)
    print('seed file created')
    print('number of particles: ',n)

#edit the config
ftlesimname = f"PyLag_FTLE_{simname}"
cf.read(pcfg)

cf.set('GENERAL', 'output_file',ftlesimname)

 # Set the time to start simulations 
Date = datetime.datetime(2021,6,21)
Date = datetime.datetime.combine(Date, datetime.datetime.min.time())
date_string = Date.strftime('%Y-%m-%d %H:%M:%S')
delta = timedelta(hours = 0)
Date = Date - delta
date_string = Date.strftime('%Y-%m-%d %H:%M:%S')

cf.set('SIMULATION', 'start_datetime',date_string)
print('Start time: {}'.format(cf.get('SIMULATION', 'start_datetime')))

# Specify that this is a reverse tracking experiment
cf.set('SIMULATION', 'time_direction','reverse')
print('Time direction: {}'.format(cf.get('SIMULATION', 'time_direction')))

#set the path to the initial positions file
cf.set('SIMULATION','initial_positions_file', ftle_seedfile)
print('Seedfile: {}'.format(cf.get('SIMULATION', 'initial_positions_file')))
# We will do an ensemble run

# set duration of each ensemble release  
cf.set('SIMULATION', 'duration_in_days','0.5')
print('Duration of each release (hours): {}'.format(cf.getfloat('SIMULATION', 'duration_in_days')*24))

cf.set('NUMERICS', 'iterative_method', 'Adv_RK4_3D')

cf.set('OCEAN_CIRCULATION_MODEL','has_is_wet'  ,'True')
cf.set('OCEAN_CIRCULATION_MODEL','horizontal_eddy_viscosity_constant' ,'10.0')
cf.set('OCEAN_CIRCULATION_MODEL','time_dim_name'  ,'time')
cf.set('OCEAN_CIRCULATION_MODEL','depth_dim_name'  ,'depth')
cf.set('OCEAN_CIRCULATION_MODEL','latitude_dim_name'  ,'lat')
cf.set('OCEAN_CIRCULATION_MODEL','longitude_dim_name'  ,'lon')
cf.set('OCEAN_CIRCULATION_MODEL','time_var_name'  ,'time')
cf.set('OCEAN_CIRCULATION_MODEL','uo_var_name'  ,'u')
cf.set('OCEAN_CIRCULATION_MODEL','vo_var_name'  ,'v')
cf.set('OCEAN_CIRCULATION_MODEL','wo_var_name'  ,'ww')
cf.set('OCEAN_CIRCULATION_MODEL','zos_var_name'  ,'zeta')
cf.set('OCEAN_CIRCULATION_MODEL','kh_var_name'  ,'')
cf.set('OCEAN_CIRCULATION_MODEL','ah_var_name'  ,'')
cf.set('OCEAN_CIRCULATION_MODEL','thetao_var_name'  ,'temp')
cf.set('OCEAN_CIRCULATION_MODEL','so_var_name'  ,'salinity')

# Save a copy in the simulation directory
with open(f"{simulation_dir}/pylag_ftle_{simname}.cfg", 'w') as config:
    cf.write(config)
pcfg = f'{simulation_dir}/pylag_ftle_{simname}.cfg'    

###################
#  setup the run  #
###################

# Change to the run directory and launch
os.chdir(simulation_dir)
print(f'running pylag for ftle in {simulation_dir}')

# write submit script 
with open(f"ftlerun_{simname}.sh", "w") as f:
    f.write("#!//usr/bin/env bash\n")
    f.write("set dir .\n")
 #  f.write(f"python -m pylag.main -c {pcfg}\n") # for serial applications
    f.write(f"mpiexec -np {cores} python -m pylag.parallel.main -c {pcfg}\n") # for parallel applications
    
os.chmod(f"ftlerun_{simname}.sh", stat.S_IRWXU)



In [ ]:
#run pylag for ftle
print('running pylag for FTLE/LCS')
try:
    subprocess.call([f'./ftlerun_{simname}.sh'])
except:
    print('Run failed.')
    pass
print('Pylag for FTLE/LCS ran sucessfully')

In [ ]:
########################
# running MyCOAST_FTLE #
########################


fileroot = f'{simulation_dir}/PyLag_FTLE_{simname}'

# write submit script 
with open(f"{simulation_dir}/run-ftle{simname}.sh", "w") as f:
    f.write("#!//usr/bin/env bash\n")
    f.write("set dir .\n")
    f.write("echo '>>>> FTLE <<<<<'\n")
    f.write(f"python -m MYCOASTLCS -j {ftlejson} -i '{fileroot}*.nc' -o Pylag_.nc\n")

    #make the script executable
os.chmod(f"{simulation_dir}/run-ftle{simname}.sh", 0o755)



In [ ]:
#run the mycoast tool
print('running the mycoastlcs tool')
try:
    subprocess.call([f'./run-ftle{simname}.sh'])
except:
    print('Run failed.')
    pass
print('mycoastlcs tool ran sucessfully')

In [ ]:
from netCDF4 import Dataset
# plots with ftle and lcs

print('Plotting CSO and LCS fields at midday') 
from matplotlib import animation, rc
import netCDF4
from IPython.display import HTML

ftlefile = f'{simulation_dir}/Pylag_ftle.nc'

# Dataset holding FTLE/LCS fields
with Dataset(ftlefile) as nc:
    lon = nc.variables['x0'][:]
    lat = nc.variables['y0'][:]
    LCS = nc.variables['LCS_backward'][:, :, :]
    FTLE = nc.variables['FTLE_backward'][:, :, :]
    nctime = nc.variables['time']
    numtime = nctime[:]
    time = netCDF4.num2date(numtime, nctime.units)

blanks = np.zeros_like(bathy)
# apply mask to LCS fields as they are in the FTLE
LCS_masked = np.ma.masked_array(LCS,mask=FTLE.mask)
lon,lat = lonlat_from_utm(lon,lat,epsg_code='32630')
LCSsum = np.sum(LCS_masked,axis = 0)
LCSsum = LCSsum.astype('float')
LCSsum[LCSsum == 0] = 'nan'

colormap=plt.get_cmap('Greys_r')

fig, ax = create_figure(figure_size=(26., 26.), projection=ccrs.PlateCarree(), font_size=font_size, bg_color='grey')

empty = np.zeros_like(blanks)
empty[:] = np.nan
plotter.plot_field(ax, blanks, extents=plt_lims, add_colour_bar=False,cmap = colormap,vmin=-60,vmax=0,)
colormap=plt.get_cmap('cool')
plot = ax.pcolormesh(lon, lat, LCSsum[:,:],  cmap=colormap, transform=ccrs.PlateCarree(),animated=True,vmin=0,vmax=10)
pos = ax.get_position().get_points()
cax = fig.add_axes([pos[1,0]+0.01, pos[0,1], 0.02, pos[1,1] - pos[0,1]])
cbar = fig.colorbar(plot, cax=cax)
cbar.ax.tick_params(labelsize=font_size)
cbar.set_label('LCS field', fontsize=font_size)


# Plot particle final positions
ax.scatter(csolons, csolats, s=12, color='k', edgecolors='none')

# plot CSO sites
for i in range(len(CSOlon)):
    if spill_bool[i] == True:
        ax.scatter(CSOlon[i], CSOlat[i], s=70, color='r', edgecolors='k')
    else:
        ax.scatter(CSOlon[i], CSOlat[i], s=50, marker = 'x', color='k')

plotter.set_title(ax, f'Target: 2021-06-21 12:00:00  n = {numpar}')

#plot = ax.pcolormesh(lon, lat, LCSsum[:,:],  cmap=colormap)
cawsand = shp.Reader(f'{shp_dir}/cawsand.shp')
for shape in cawsand.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.plot(x,y,'purple')
    
bovisand = shp.Reader(f'{shp_dir}/bovisand.shp')
for shape in bovisand.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.plot(x,y,'yellow')

firestone = shp.Reader(f'{shp_dir}/firestone.shp')
for shape in firestone.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.plot(x,y,'red')
       
tinside = shp.Reader(f'{shp_dir}/tinside.shp')
for shape in tinside.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    ax.plot(x,y,'green')
    
legend_elements = [Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='yellow', markeredgecolor= 'yellow', markersize=7, label = 'Bovisand Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='green', markeredgecolor= 'g', markersize=7, label = 'Tinside East'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='red', markeredgecolor= 'r', markersize=7, label = 'Firestone Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='purple', markeredgecolor= 'purple', markersize=7, label = 'Kingsand Bay'),
                      Line2D([0], [0], marker = 'o', color = 'w', markerfacecolor='red', markeredgecolor= 'black', markersize=15, label = 'Active CSOs'),
                      Line2D([0], [0], marker = 'x', color = 'w', markerfacecolor='black', markeredgecolor= 'black', markersize=10, label = 'Inactive CSOs'),
                      Line2D([0], [0], marker = '.', color = 'w', markerfacecolor='black', markeredgecolor= 'black', markersize=10, label = 'CSO particles')]


ax.legend(handles=legend_elements, loc='best',framealpha=1)
plt.savefig(f'{simulation_dir}/PlymouthSound_CSOLCS_{simname}.png')
#plt.close()

In [ ]:

print('Generating FTLE animations....')
import cmocean.cm as cmo
rc('animation', html='html5')
blanks2 = blanks.data
blanks2[:] = 0

ftlemax = np.max(FTLE[:])
ftlemin = np.min(FTLE[:])
#### plotting the FTLE fields ###
fig, ax = plt.subplots(1, 1, figsize=(20., 15.))
font_size = 22
#fig, ax = create_figure(figure_size=(26., 26.), font_size=font_size, bg_color='white')
#plotter.plot_field(ax, blanks2, extents=plt_lims, add_colour_bar=False,vmin=0, vmax=1)
#colormap=plt.get_cmap('binary')
colormap = cmo.balance
colormap = 'PuBu'
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 22})
plot = ax.pcolormesh(lon, lat, FTLE[-1,:,:].squeeze(),  cmap=colormap,animated=True,vmin=ftlemin, vmax=ftlemax,shading='gouraud')
ax.set_facecolor('grey')
plt.xlabel('Longitude',fontsize=font_size)
plt.ylabel('Latitude',fontsize=font_size)
pos = ax.get_position().get_points()
cax = fig.add_axes([pos[1,0]+0.01, pos[0,1], 0.02, pos[1,1] - pos[0,1]])
cbar = fig.colorbar(plot, cax=cax)
cbar.ax.tick_params(labelsize=font_size)
cbar.set_label('FTLE field', fontsize=font_size)
fig.suptitle(f'FTLE fields at midday')

#gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=1, color='gray', alpha=0.5, linestyle='-')
#gl.top_labels = False
#gl.right_labels = False
#gl.xlines = False
#gl.ylines = False
plt.savefig(f'{simulation_dir}/ftle_surface.png')




In [ ]:

###################################################
# animating the time evolution of the FTLE fields #
###################################################

def init():
#    plot.set_array([], mask=True)#
    return (plot,)


def animate(i):
    data = FTLE[i,:,:].astype(float)
    fig.suptitle('Target: %s ' % (time[i].strftime('%Y-%m-%d %H:%M')))
    plot = ax.pcolormesh(lon, lat, data.squeeze(),  cmap=colormap,animated=True,vmin=ftlemin,vmax=ftlemax,shading='gouraud')
    pos = ax.get_position().get_points()
    cax = fig.add_axes([pos[1,0]+0.01, pos[0,1], 0.02, pos[1,1] - pos[0,1]])
    cbar = fig.colorbar(plot, cax=cax)
    cbar.ax.tick_params(labelsize=font_size)
    cbar.set_label('FTLE field', fontsize=font_size)
    return (plot,)

# Prevent the basic figure from being plotted too
plt.close(fig)

anim = animation.FuncAnimation(fig, animate,  frames=len(FTLE[:,1,1]), interval=100, blit=True)
anim.save(f'{simulation_dir}/FTLEAnimation.gif', writer='imagemagick', fps=5)

HTML(anim.to_jshtml())


In [ ]:
from matplotlib import ticker

print('Generating LCS animations....')
fig, ax = plt.subplots(1, 1, figsize=(20., 15.))

colormap = 'Blues'
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 22})
plot = ax.pcolormesh(lon, lat, LCS_masked[-1,:,:].squeeze(),  cmap=colormap,animated=True,shading='gouraud')
ax.set_facecolor('grey')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
fig.suptitle(f'LCS fields at midday')
plt.savefig(f'{simulation_dir}/lcs_surface.png')


In [ ]:

###################################################
# animating the time evolution of the LCS fields #
###################################################

def init():
#    plot.set_array([], mask=True)#
    return (plot,)


def animate(i):
    data = LCS_masked[i,:,:].astype(float)
    fig.suptitle('Target: %s ' % (time[i].strftime('%Y-%m-%d %H:%M')))
    plot = ax.pcolormesh(lon, lat, data.squeeze(),  cmap=colormap,animated=True,shading='gouraud')
    return (plot,)

# Prevent the basic figure from being plotted too
plt.close(fig)

anim = animation.FuncAnimation(fig, animate,  frames=len(LCS_masked[:,1,1]), interval=100, blit=True)
anim.save(f'{simulation_dir}/LCSAnimation.gif', writer='imagemagick', fps=5)

HTML(anim.to_jshtml())


In [ ]:
#calculate the daily LCS at each site

print('calculating LCS within Plymouth Sound bathing areas')
from shapely.geometry import Point, Polygon
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface
import csv

LCSlons = []
LCSlats = []

for i in range(0,len(time)-1): #time
    lats = []
    lons = []
    for j in range(0,len(lon)-1): #lon
        for k in range(0,len(lat)-1): #lat
            if LCS_masked[i,j,k] == 1:

                lons.append(lon[j])
                lats.append(lat[k])
                
            else:
                continue

    #time series lists of coordinates where LCS_masked == 1
    LCSlons.append(lons)
    LCSlats.append(lats)
    

bovisandpoly = bovisand.shapes() 
tinsidepoly = tinside.shapes() 
firestonepoly = firestone.shapes() 
cawsandpoly = cawsand.shapes() 



In [ ]:
bovi_timeevo = []
tins_timeevo = []
caws_timeevo = []
fire_timeevo = []

for t in range(len(LCSlons)):
    
    tlonsarray = np.asarray(LCSlons[t])
    tlatsarray = np.asarray(LCSlats[t])
    
    tbovi = 0
    for i in range(len(tlonsarray)):
        point=Point(tlonsarray[i],tlatsarray[i])
        if point.within(shape(bovisandpoly)): 
            tbovi = tbovi+1
            
    ttins = 0
    for i in range(len(tlonsarray)):
        point=Point(tlonsarray[i],tlatsarray[i])
        if point.within(shape(tinsidepoly)): 
            ttins = ttins+1
            
    tfire = 0
    for i in range(len(tlonsarray)):
        point=Point(tlonsarray[i],tlatsarray[i])
        if point.within(shape(firestonepoly)): 
            tfire = tfire+1
    
    tcaws = 0
    for i in range(len(tlonsarray)):
        point=Point(tlonsarray[i],tlatsarray[i])
        if point.within(shape(cawsandpoly)): 
            tcaws = tcaws+1
            
    bovi_timeevo.append(tbovi)
    tins_timeevo.append(ttins)
    fire_timeevo.append(tfire)
    caws_timeevo.append(tcaws)

bovi_timeevo = np.asarray(bovi_timeevo)
tins_timeevo = np.asarray(tins_timeevo)
fire_timeevo = np.asarray(fire_timeevo)
caws_timeevo = np.asarray(caws_timeevo)

bovisandlcscount = sum(bovi_timeevo)
tinsidelcscount = sum(tins_timeevo)
firestonelcscount = sum(fire_timeevo)
cawsandlcscount = sum(caws_timeevo)


bovisandlcskm = float(bovisandlcscount)/float(bovikm)
tinsidelcskm = float(tinsidelcscount)/float(tinskm)
firestonelcskm = float(firestonelcscount)/float(firekm)
cawsandlcskm = float(cawsandlcscount)/float(cawskm)



In [ ]:

#plot the time evolution of the LCS fields at each bathing site 

maxy = np.maximum.reduce([bovi_timeevo,tins_timeevo,fire_timeevo,caws_timeevo])
maxy = np.max(maxy)

fig, axs = plt.subplots(4,sharex=True,figsize=(10,15),facecolor='white')
fig.suptitle('2021-06-21',fontsize = 20)
fig.tight_layout(rect=[0, 0.03, 1, 0.95]) 

# Set common labels
fig.text(0.5, 0, 'Time Since Midnight', ha='center', va='center',fontsize = 20)
fig.text(-0.05, 0.5, 'LCS concentration', ha='center', va='center',rotation='vertical',fontsize = 20)

time_plt = list(range(1, 24))
axs[0].plot(time_plt, fire_timeevo,'r')
axs[1].plot(time_plt, tins_timeevo,'g')
axs[2].plot(time_plt, caws_timeevo,'purple')
axs[3].plot(time_plt, bovi_timeevo,'y')


axs[0].set(ylim=(0,maxy),title='Firestone Bay')
axs[1].set(ylim=(0,maxy),title='Tinside East')
axs[2].set(ylim=(0,maxy),title='Cawsand Bay')
axs[3].set(ylim=(0,maxy),title='Bovisand Bay')


plt.xticks(np.arange(min(time_plt)+1, max(time_plt),2))
plt.show()
fig.savefig(f'{simulation_dir}/LCS_timeevo.png', facecolor=fig.get_facecolor(), transparent=True)

In [ ]:
#save cso counts and LCS counts in csv
today = '2021-06-21'
print('saving CSO and LCS metrics to output table')
boviT = f'{tables}/bovisand.csv'
fields=[today,bovisandcsokm,bovisandlcskm]
with open(boviT, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    
fireT = f'{tables}/firestone.csv'
fields=[today,firestonecsokm,firestonelcskm]
with open(fireT, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    
tinsT = f'{tables}/tinside.csv'
fields=[today,tinsidecsokm,tinsidelcskm]
with open(tinsT, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    
cawsT = f'{tables}/cawsand.csv'
fields=[today,cawsandcsokm,cawsandlcskm]
with open(cawsT, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    
print('Pollution risk tool completed for ',today)
print('fin!')